In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV,GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import plot_roc_curve

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Crime Dataset/Crimes_-_2001_to_Present.csv',nrows=100)

In [4]:
df.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12,61,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29,25,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8,44,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35,21,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28,25,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [5]:
 df.dropna(inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 99
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    96 non-null     int64  
 1   Case Number           96 non-null     object 
 2   Date                  96 non-null     object 
 3   Block                 96 non-null     object 
 4   IUCR                  96 non-null     object 
 5   Primary Type          96 non-null     object 
 6   Description           96 non-null     object 
 7   Location Description  96 non-null     object 
 8   Arrest                96 non-null     bool   
 9   Domestic              96 non-null     bool   
 10  Beat                  96 non-null     int64  
 11  District              96 non-null     int64  
 12  Ward                  96 non-null     int64  
 13  Community Area        96 non-null     int64  
 14  FBI Code              96 non-null     object 
 15  X Coordinate          96 

In [7]:
df = df.dropna(subset=['X Coordinate', 'Y Coordinate','Latitude','Longitude','Location'], how='all')

In [8]:
df.drop('Case Number',inplace=True,axis=1)
df.drop('IUCR',inplace=True,axis=1)
df.drop('FBI Code',inplace=True,axis=1)
df.drop('X Coordinate',inplace=True,axis=1)
df.drop('Y Coordinate',inplace=True,axis=1)
df.drop('Location',inplace=True,axis=1)

In [9]:
df.head(5)

,ID,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Updated On,Latitude,Longitude
0,10224738,09/05/2015 01:30:00 PM,043XX S WOOD ST,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,924,9,12,61,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000
1,10224739,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,THEFT,POCKET-PICKING,CTA BUS,False,False,1511,15,29,25,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400
3,10224740,09/05/2015 12:45:00 PM,035XX W BARRY AVE,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,1412,14,35,21,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650
4,10224741,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,ASSAULT,SIMPLE,APARTMENT,False,True,1522,15,28,25,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121
5,10224742,09/05/2015 10:55:00 AM,082XX S LOOMIS BLVD,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,614,6,21,71,2015,02/10/2018 03:50:01 PM,41.744379,-87.658431


In [10]:
def month_preprocess(x):
    return int(x.split('/')[0])
df['MOF'] = df['Date'].apply(month_preprocess);
df.drop('Date',inplace=True,axis=1)

In [11]:
numerical_features = ['Beat', 'Ward','MOF']

In [12]:
categorical_features = ['Description','Location Description', 'Domestic','Block', 'Arrest','Primary Type']
encoders = {}

In [13]:
numerical_features = df._get_numeric_data().columns

In [14]:
categorical_features = list(set(df.columns) - set(numerical_features))

In [15]:
for each in categorical_features:
    encoders[each] = LabelEncoder()
    df[each] = encoders[each].fit_transform(df[each])

In [16]:
df.head(5)

,ID,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Updated On,Latitude,Longitude,MOF
0,10224738,51,1,8,14,False,True,924,9,12,61,2015,0,41.815117,-87.670000,9
1,10224739,13,11,20,4,False,False,1511,15,29,25,2015,0,41.895080,-87.765400,9
3,10224740,46,7,24,19,True,False,1412,14,35,21,2015,0,41.937406,-87.716650,9
4,10224741,0,0,28,1,False,True,1522,15,28,25,2015,0,41.881903,-87.755121,9
5,10224742,80,2,9,14,False,False,614,6,21,71,2015,0,41.744379,-87.658431,9


In [47]:
X = df.drop(['Primary Type','Arrest','Domestic'],axis=1)
y = df['Primary Type']

In [48]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    96 non-null     int64  
 1   Block                 96 non-null     int64  
 2   Description           96 non-null     int64  
 3   Location Description  96 non-null     int64  
 4   Beat                  96 non-null     int64  
 5   District              96 non-null     int64  
 6   Ward                  96 non-null     int64  
 7   Community Area        96 non-null     int64  
 8   Year                  96 non-null     int64  
 9   Updated On            96 non-null     int64  
 10  Latitude              96 non-null     float64
 11  Longitude             96 non-null     float64
 12  MOF                   96 non-null     int64  
dtypes: float64(2), int64(11)
memory usage: 10.5 KB


In [49]:
y.head()

0     1
1    11
3     7
4     0
5     2
Name: Primary Type, dtype: int64

In [50]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(X,y,test_size =0.2)

In [51]:
x_train_std = std.fit_transform(x_train)
x_test_std = std.transform(x_test)
print(x_train_std)
print(x_test_std)

[[-1.15307732e-01  1.57807527e+00  1.00428442e+00  4.40581364e-02
  -9.06855629e-01 -8.88995186e-01 -8.98178946e-01  7.08370713e-01
  -9.03507903e-02 -2.58624961e-01 -1.59131663e+00  7.83342637e-01
   1.99964454e-01]
 [-1.15312635e-01  8.31289962e-01 -5.39379720e-01 -1.76966848e+00
   2.30415014e+00  2.32067864e+00  5.29887997e-01 -8.01054732e-01
  -9.03507903e-02 -2.58624961e-01  9.34588533e-01 -1.83922339e+00
   1.99964454e-01]
 [-1.15744094e-01 -1.63310156e+00  7.47007060e-01 -1.76966848e+00
   7.14702284e-01  7.15841729e-01  4.58484650e-01 -4.99169643e-01
  -9.03507903e-02 -2.58624961e-01  4.78329189e-01 -1.40923214e+00
   1.99964454e-01]
 [-1.15386179e-01  6.07254369e-01  1.26156177e+00  1.02068016e+00
   1.01814233e+00  1.03680911e+00  1.24392147e+00 -1.10293982e+00
  -9.03507903e-02 -2.58624961e-01  1.66008944e+00 -7.13558099e-01
   1.99964454e-01]
 [-1.15611715e-01  1.20468262e+00 -1.10584127e-01  1.16019759e+00
  -1.05295639e+00 -1.04947888e+00 -1.04098564e+00  4.06485624e-01


In [52]:
from sklearn.svm import SVC
sv = SVC(probability=True)
sv.fit(x_train_std, y_train)

SVC(probability=True)

In [53]:
y_pred = sv.predict(x_test)
ac_rf = accuracy_score(y_test,y_pred)
print(f"Accuracy of support vector classifier in total dataset: {ac_rf}")

Accuracy of support vector classifier in total dataset: 0.45


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [54]:
from sklearn.metrics import classification_report , confusion_matrix, accuracy_score 
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         2
          11       0.45      1.00      0.62         9

    accuracy                           0.45        20
   macro avg       0.06      0.12      0.08        20
weighted avg       0.20      0.45      0.28        20



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
X.columns

Index(['ID', 'Block', 'Primary Type', 'Description', 'Location Description',
       'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'Year',
       'Updated On', 'Latitude', 'Longitude', 'MOF'],
      dtype='object')

In [58]:
block = "102XX S VERNON AVE"
location_description = "STREET"
secondary_descrption = "TO VEHICLE"
arrest = "N"
domestic = "Y"
beat = 511
ward = 9
latitude = 41.708514886
longitude = -87.61258026
mof = 5

In [59]:
columns = ['Block','Description', 'Location Description', 'Beat', 'Ward','Latitude', 'Longitude', 'MOF']

In [60]:
categorical_features_for_predict = ['Description','Location Description', 'Domestic','Block', 'Arrest',]

In [61]:
def process_input(x):
    x = pd.DataFrame(x, columns=columns)
    for each in categorical_features_for_predict:
        print(each)
        x[each] = encoders[each].transform(x[each])
    x = std.transform(x)
    return x
        
x = [[block,secondary_descrption, location_description, beat,ward,latitude, longitude,mof]]
x = process_input(x)  

Description
Location Description
Domestic


KeyError: ignored

In [62]:
y1 = sv.predict(x)
print(y1)
results = sv.predict_proba(x)
print(results)

ValueError: ignored